In [1]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

In [2]:
import time
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
start = time.time()
rf_clf.fit(X_train, y_train)
end = time.time()
print(f'{end - start} sec')

37.2671685218811 sec


In [3]:
from sklearn.metrics import accuracy_score

y_pred = rf_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9705

In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
X_train_reduced = pca.fit_transform(X_train)

In [5]:
rf_clf2 = RandomForestClassifier(n_estimators=100, random_state=42)
start = time.time()
rf_clf2.fit(X_train_reduced, y_train)
end = time.time()
print(f'{end - start} sec')

76.1855537891388 sec


In [6]:
X_test_reduced = pca.transform(X_test)
y_pred = rf_clf2.predict(X_test_reduced)
accuracy_score(y_test, y_pred)

0.9481